# This script is for embedding classifcation training (Using BERT Embedding)

In [1]:
import os
import pandas as pd
# from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
# sklearn.metrics.accuracy_score
# import xgboost as xgb
import numpy as np
import itertools 
import tqdm
from time import sleep
import random
# from docx import Document
# import docx2txt

In [2]:
import torch

from transformers import BertTokenizer, BertForSequenceClassification
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
from accelerate.utils.memory import clear_device_cache
from torch.optim import AdamW
from transformers import get_scheduler
from accelerate.test_utils.testing import get_backend
from tqdm.auto import tqdm
import evaluate

In [3]:
import warnings 
warnings.filterwarnings("ignore")

In [4]:
# This is just for disabling warning. If you are debugging bypass this cell
from transformers import logging

# logging.set_verbosity_warning()
logging.set_verbosity_error() 

In [5]:
from datasets import Dataset
# Prepare the model input features
def tokenize(batch):
    return tokenizer(batch["summarization"], padding=True, truncation=True)

In [6]:
# Please specify your model here
device, _, _ = get_backend()
model_name = "bert-base-uncased"
# model_name = "/data/project/deberta-v3-base"
if "bert" in model_name.split("-"):
    tokenizer = BertTokenizer.from_pretrained(model_name)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
# mannually generate the search grid
def makeGrid(pars_dict):  
    keys=pars_dict.keys()
    combinations=itertools.product(*pars_dict.values())
    ds=[dict(zip(keys,cc)) for cc in combinations]
    return ds

## Data Preparation

In [8]:
df_raw = pd.read_excel("../Dataset/MergedDataset_231207_ForElly_Excel.xlsx")

# Remove "." in the feature column
row_num, column_num = df_raw.shape
for rowID in range(row_num):
    for columnID in range(column_num):
        if "." == df_raw.iloc[rowID, columnID]:
            df_raw.iloc[rowID, columnID] = np.nan

# prepare the feature list
featureList = []
for item in df_raw.columns:
    if item.startswith("Item"):
        featureList.append(item)

patientList = []
roundList = []
for idx, row in df_raw.iterrows():
    patient = int(str(row["IAPTus_Num"]).split("_")[0])
    if "_" in str(row["IAPTus_Num"]):
        Round = int(str(row["IAPTus_Num"]).split("_")[1])
    else:
        Round = 0
    patientList.append(patient)
    roundList.append(Round)
df_raw["patient"] = patientList
df_raw["round"] = roundList

In [9]:
df_referral = pd.read_json("../Dataset/ReferralLetterSummary.json", lines=True)

In [10]:
df = pd.merge(df_raw, df_referral, on=["patient","round"])

# Label Preparation

## Here I use RecoveryDesc, not the EndDesc as the prediction target. EndDesc has 4 label categories and Referral Letter has only 44 data points, so it does not meet the conditions for tuning.

In [11]:
df = df[df["RecoveryDesc"] == df["RecoveryDesc"]]
df = df[(df["RecoveryDesc"].isin(["At recovery", "Not at recovery"]))]
df.reset_index().drop("index", axis=1)
print(df.shape)

labelList = []
for idx, row in df.iterrows():
    if (row["ReliableChangeDesc"] == "Reliable improvement") & (row["ReliableRecoveryDesc"] == "Reliable recovery") & (row["RecoveryDesc"] == "At recovery"):
        labelList.append(1)
    else:
        labelList.append(0)
df["labels"] = np.array(labelList, dtype=float)
labelList = ["labels"]

(43, 285)


In [12]:
df = df[["IAPTus_Num_x","patient","round","text", "summarization", "labels"]]
df = df.reset_index().drop("index", axis=1)

In [13]:
# Keep 10% data as the test data
train, test = train_test_split(df, test_size=0.1)
train.shape

(38, 6)

## Training Code

In [14]:
def train_predict(df, param, model_name):
    # KFold Split
    skf = StratifiedKFold(n_splits=5)
    print(skf)
    FoldList = []
    for i, (train_index, val_index) in enumerate(skf.split(df["summarization"], df["labels"])):
        train = df.iloc[train_index]
        val = df.iloc[val_index]
        
        train = train[["summarization", "labels"]]
        trainDataset = Dataset.from_pandas(train, preserve_index=False).map(tokenize, batched=True, batch_size=None)
        trainDataset.set_format("torch")

        val = val[["summarization", "labels"]]
        valDataset = Dataset.from_pandas(val, preserve_index=False).map(tokenize, batched=True, batch_size=None)
        valDataset.set_format("torch")

        # Now I only have 16G GPU Memory, so for BERT the biggest batch size I can reach is 32, while for DeBERTa it is 4. 
        # We'd better find a machine with bigger GPU memory to run DeBERTa with batch size 32
        if "bert" in model_name.split("-"):
            train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=32)
        else:
            # train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=32)
            train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=4)
        eval_dataloader = DataLoader(valDataset, batch_size=1)
        aucList = []
        for _ in range(5):
            aucList.append(singleTrain(train_dataloader, eval_dataloader, param, model_name))
        
        FoldList.append(aucList)
    return FoldList  

In [31]:
def singleTrain(train_dataloader, eval_dataloader, hypparam, model_name, save=None):
    # Clean GPU 
    with torch.no_grad():
        torch.cuda.empty_cache()

    if "bert" in model_name.split("-"):
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
        for param in model.bert.parameters():
            param.requires_grad = False
    else:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
    optimizer = AdamW(model.parameters(), lr=hypparam["lr"])        
    num_epochs = hypparam["epoch"]
    num_training_steps = num_epochs * len(train_dataloader)
    model.to(device)    
    model.train()
    for epoch in range(num_epochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
    
            optimizer.step()
            optimizer.zero_grad()
            # progress_bar.update(1)      

    finalResult = []
    finalLabel = []
    model.eval()    
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
        with torch.no_grad():
            outputs = model(**batch)
    
        logits = outputs.logits
        predictions = torch.sigmoid(logits)
        # print(predictions)
        finalResult += predictions.tolist()
        finalLabel += batch["labels"].tolist()
        # metric.add_batch(predictions=predictions, references=batch["labels"])
    # print(finalResult)
    # print(finalLabel)
    # print(roc_auc_score(finalLabel, finalResult))
    auc = roc_auc_score(finalLabel, finalResult)
    if save != None:
        model.save_pretrained("./tmp", from_pt=True) 
    return auc

## Test for the training code

In [20]:
# can get 100%, but not stable
# param_dict = {
#     "lr": 1e-5,
#     "epoch": 10
# }

param_dict = {
    'lr': 1e-07, 
    'epoch': 7
}

In [21]:
train = train[["summarization", "labels"]]
trainDataset = Dataset.from_pandas(train, preserve_index=False).map(tokenize, batched=True, batch_size=None)
trainDataset.set_format("torch")

test = test[["summarization", "labels"]]
testDataset = Dataset.from_pandas(test, preserve_index=False).map(tokenize, batched=True, batch_size=None)
testDataset.set_format("torch")

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [22]:
train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=32)
test_dataloader = DataLoader(testDataset, batch_size=1)

In [32]:
if "tmp" not in os.listdir("./"):
    os.mkdir("tmp")
singleTrain(train_dataloader, test_dataloader, param_dict, model_name, save="/tmp")

np.float64(1.0)

In [33]:
with torch.no_grad():
    torch.cuda.empty_cache()
model = AutoModelForSequenceClassification.from_pretrained("./tmp").to(device)
model.eval()    
finalResult = []
finalLabel = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.sigmoid(logits)
    # print(predictions)
    finalResult += predictions.tolist()
    finalLabel += batch["labels"].tolist()
print(finalResult)
print(finalLabel)

[[0.4126128852367401], [0.43854087591171265], [0.4176628589630127], [0.40840137004852295], [0.41806748509407043]]
[0.0, 1.0, 1.0, 0.0, 1.0]


# Grid Search for Hyperparameter settings

In [16]:
def MyGridSearch(df, param_dict, model_name):
    searchSpace = makeGrid(param_dict)
    print("Search Space Size:" + len(searchSpace).__str__())
    resultList = []
    for param in tqdm(searchSpace):
        #sleep(1)
        scores = train_predict(df, param, model_name)
        result = {}
        result["auc_mean"] = np.mean(scores)
        result["auc_std"] = np.std(scores)
        result["auc"] = scores
        result["param"] = param
        resultList.append(result)
    sortedResult = sorted(resultList, key=lambda x: x["auc_mean"], reverse=True)
    print(sortedResult[:5])
    return sortedResult

In [17]:
param_dict = {
    "lr": [1e-8, 5e-8, 1e-7, 5e-7, 8e-7, 1e-6, 2e-6, 5e-6, 1e-5, 2e-5, 5e-5, 1e-4],
    "epoch": [3,4,5,6,7,8,9,10,12,15,20]
}

In [18]:
finalResult = MyGridSearch(train, param_dict, model_name)

Search Space Size:132


  0%|          | 0/132 [00:00<?, ?it/s]

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

[{'auc_mean': np.float64(0.6175), 'auc_std': np.float64(0.16254059322041509), 'auc': [[np.float64(0.625), np.float64(0.5), np.float64(0.625), np.float64(0.5625), np.float64(0.6875)], [np.float64(0.625), np.float64(0.75), np.float64(0.5625), np.float64(0.8125), np.float64(0.375)], [np.float64(0.625), np.float64(0.75), np.float64(0.875), np.float64(0.75), np.float64(0.8125)], [np.float64(0.41666666666666663), np.float64(0.6666666666666666), np.float64(0.75), np.float64(0.3333333333333333), np.float64(0.75)], [np.float64(0.5), np.float64(0.5), np.float64(0.25), np.float64(0.8333333333333333), np.float64(0.5)]], 'param': {'lr': 1e-07, 'epoch': 7}}, {'auc_mean': np.float64(0.5941666666666666), 'auc_std': np.float64(0.17971041520550035), 'auc': [[np.float64(0.5625), np.float64(0.75), np.float64(0.4375), np.float64(0.5), np.float64(0.4375)], [np.float64(0.6875), np.float64(0.75), np.float64(0.5625), np.float64(0.625), np.float64(0.6875)], [np.float64(0.5), np.float64(0.6875), np.float64(0.625

In [19]:
finalResult

[{'auc_mean': np.float64(0.6175),
  'auc_std': np.float64(0.16254059322041509),
  'auc': [[np.float64(0.625),
    np.float64(0.5),
    np.float64(0.625),
    np.float64(0.5625),
    np.float64(0.6875)],
   [np.float64(0.625),
    np.float64(0.75),
    np.float64(0.5625),
    np.float64(0.8125),
    np.float64(0.375)],
   [np.float64(0.625),
    np.float64(0.75),
    np.float64(0.875),
    np.float64(0.75),
    np.float64(0.8125)],
   [np.float64(0.41666666666666663),
    np.float64(0.6666666666666666),
    np.float64(0.75),
    np.float64(0.3333333333333333),
    np.float64(0.75)],
   [np.float64(0.5),
    np.float64(0.5),
    np.float64(0.25),
    np.float64(0.8333333333333333),
    np.float64(0.5)]],
  'param': {'lr': 1e-07, 'epoch': 7}},
 {'auc_mean': np.float64(0.5941666666666666),
  'auc_std': np.float64(0.17971041520550035),
  'auc': [[np.float64(0.5625),
    np.float64(0.75),
    np.float64(0.4375),
    np.float64(0.5),
    np.float64(0.4375)],
   [np.float64(0.6875),
    np.flo

In [ ]:
param_dict = {
    "lr": 1e-5,
    "epoch": 10
}

In [18]:
def singleTrain(train_dataloader, eval_dataloader, hypparam, model_name, save=None):
    # Clean GPU 
    with torch.no_grad():
        torch.cuda.empty_cache()

    if "bert" in model_name.split("-"):
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
        for param in model.bert.parameters():
            param.requires_grad = False
    else:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
    optimizer = AdamW(model.parameters(), lr=hypparam["lr"])        
    num_epochs = hypparam["epoch"]
    num_training_steps = num_epochs * len(train_dataloader)
    model.to(device)    
    model.train()
    for epoch in range(num_epochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
    
            optimizer.step()
            optimizer.zero_grad()
            # progress_bar.update(1)      

    finalResult = []
    finalLabel = []
    model.eval()    
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
        with torch.no_grad():
            outputs = model(**batch)
    
        logits = outputs.logits
        predictions = torch.sigmoid(logits)
        # print(predictions)
        finalResult += predictions.tolist()
        finalLabel += batch["labels"].tolist()
        # metric.add_batch(predictions=predictions, references=batch["labels"])
    # print(finalResult)
    # print(finalLabel)
    # print(roc_auc_score(finalLabel, finalResult))
    auc = roc_auc_score(finalLabel, finalResult)
    if save != None:
        model.save_pretrained(save) 
    return auc

In [29]:
if "tmp" not in os.listdir("./"):
    os.mkdir("tmp")
singleTrain(train_dataloader, test_dataloader, param_dict, model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IndexError: too many indices for tensor of dimension 1